# Getting NLTK for Text Processing

This notebook introduced the Natural Language Toolkit (NLTK) which facilitates a braod range of tasks for text processing and representing results. It's part of the [The Art of Literary Text Analysis](ArtOfLiteraryTextAnalysis.ipynb) and assumes that you've already worked through previous notebooks ([Getting Setup](GettingSetup.ipynb), [Getting Started](GettingStarted.ipynb) and [Getting Texts](GettingTexts.ipynb)). In this notebook we'll look in particular at

* installing the nltk library (for text processing)
* simple tokenization of words
* producing a simple table of frequencies of words
* applying a list of stopwords (words to ignore)
* producing a simple concordance for a keyword

## Installing the NLTK Library

To save some time, the NLTK library should already be installed, along with many other supporting packages Python can use.

To test it, let's try to import the NLTK library. If you get an error running the cell below, let the instructor know (note the import may take a few seconds).

In [ ]:
import nltk

## Text Processing

Now that we have NLTK installed, let's use it for text processing.

**Important: you have two choices here**. If you have just completed the [Getting Texts](GettingTexts.ipynb) notebook and have saved a local copy of the file ```goldBug.txt```, then you can move down to the next cell. If not, we'll start by retrieving _The Gold Bug_ plain text that we had saved locally in the [Getting Texts](GettingTexts.ipynb) notebook:

```python
import requests
poeUrl = "http://www.gutenberg.org/cache/epub/2147/pg2147.txt"
poeString = requests.get(poeUrl).text```

And then this, in a separate cell so that we don't read repeatedly from Gutenberg:

```python
import os
# isolate The Gold Bug
start = poeString.find("THE GOLD-BUG")
end = poeString.find("FOUR BEASTS IN ONE")
goldBugString = poeString[start:end]
# save the file locally
directory = "data"
if not os.path.exists(directory):
    os.makedirs(directory)
f = open("data/goldBug.txt", "w")
f.write(goldBugString)
f.close()```

Now we should be ready to retrieve the text:

## Start here if you have data/goldBug.txt saved already

In [ ]:
f = open("data/goldBug.txt", "r")
goldBugString = f.read()
f.close()

Now we will work toward showing the top frequency words in our plain text. This involves three major steps:

1. processing our plain text to find the words (also known as tokenization)
1. counting the frequencies of each word
1. displaying the frequencies information

## Tokenization

Tokenization is the basic process of parsing a string to divide it into smaller units of the same kind. You can tokenize text into paragraphs, sentences, words or other structures, but here we're focused on recognizing words in our text. For that, let's import the ```nltk``` library and use its convenient ```word_tokenize()``` function. NLTK actually has several ways of tokenizing texts, and for that matter we could write our own code to do it. We'll have a peek at the first ten tokens.

In [ ]:
import nltk

goldBugTokens = nltk.word_tokenize(goldBugString)
goldBugTokens[:10]

We can see from the above that ```word_tokenize``` does a useful job of identifying words (including hyphenated words like "GOLD-BUG", but also includes tokens like the exclamation mark. In some cases punctuation like this might be useful, but in our case we want to focus on word frequencies, so we should filter out punctuation tokens.

To accomplish the filtering we will use a construct called a [list comprehension](https://docs.python.org/3.4/tutorial/datastructures.html#list-comprehensions) with a conditional test built in. Let's take it one step at a time, first using a loop structure like we've already seen in [Getting Texts](GettingTexts.ipynb), and then doing the same thing with a list comprehension.

In [ ]:
# loop
loopList = [] # create an empty list
for word in goldBugTokens[:10]:
    loopList.append(word) # append each word to the end of the list
print(loopList)
    
    
# list comprehension
# return the word for each word in the list goldBugTokens, up to the 10th word
print([word for word in goldBugTokens[:10]])

Identical! So the general form of a list comprehension (which is very compact) is: 

> [_expression(item)_ for _item_ in _list_)]

We can now go a step further and add a condition to the list comprehension: we'll only include the word in the final list if any of the characters in the word is alphabetic as defined by the [isalpha()](https://docs.python.org/3.4/library/stdtypes.html?highlight=isalpha#str.isalpha) function. The ```any()``` function simply confirms that at least one of the characters in the word is a list. Don't worry if this seems too complex right now - it will make sense with some experience! 

In [ ]:
print([word for word in goldBugTokens[:10] if any(c.isalpha() for c in word)])

We've now stripped out the exclamation mark (while keeping other word tokens that contain punctuation like a hyphen).

Note that our syntax above is a bit tricky since we now have two list comprehensions, first for the words, and then for the characters in the ```any()``` condition. We could perform a similar test without a second list comprehension – using a regular expression to find word characters, for instance – but this has the benefit of showing how list comprehension can work on both lists (like word tokens) and strings (a sequence of characters). It would also be possible to write this out over a few lines in 'pseudocode', in order to make each step clearer, eg:

1. for each word in goldBugTokens (up to the 10th)
2. look at each character in the word
3. return each character only if it is an alphabetic letter
4. if (3) is True, ie some alphabetic letters are present, then return the resulting word at its position in the list

## Word Frequencies

Now that we've had a first pass at word tokenization (keeping only word tokens), let's look at counting word frequencies. Essentially we want to go through the tokens and tally the number of times each one appears. Not surprisingly, the NLTK has a very convenient method for doing just this, which we can see in this small sample (the first 10 word tokens):

In [ ]:
goldBugRealWordTokensSample = [word for word in goldBugTokens[:10] if any(c.isalpha() for c in word)]
goldBugRealWordFrequenciesSample = nltk.FreqDist(goldBugRealWordTokensSample)
goldBugRealWordFrequenciesSample

This ```FreqDist``` object is a kind of dictionary, where each word is paired with its frequency (separated by a colon), and each pair is separated by a comma. This kind of dictionary also has a very convenient way of displaying results as a table:

In [ ]:
goldBugRealWordFrequenciesSample.tabulate()

The results are displayed in descending order of frequency (two occurrences of "ho"). One of the things we can notice is that "What" and "what" are calculated separately, which in some cases may be good, but for our purposes probably isn't. This might lead us to rethink our steps until now and consider the possibility of converting our string to lowercase during tokenization.

In [ ]:
goldBugTokensLowercase = nltk.word_tokenize(goldBugString.lower()) # use lower() to convert entire string to lowercase
goldBugRealWordTokensLowercaseSample = [word for word in goldBugTokensLowercase[:10] if any(c.isalpha() for c in word)]
goldBugRealWordFrequenciesSample = nltk.FreqDist(goldBugRealWordTokensLowercaseSample)
goldBugRealWordFrequenciesSample.tabulate(20)

Good, now we have "what" and "What" as the same word form counted twice. (There are disadvantages to this as well, such as more difficulty in identifying proper names and the start of sentences, but text mining is often a set of compromises.)

At this point we also notice that our variable names are getting very long. It's often preferable to use much briefer variables, and instead to add comments (lines starting with a #) to explain what the variables mean. This is something of a personal preference, as well as a tradeoff between having descriptive variable names, and readable code.

Next, let's redo our entire workflow with the full set of tokens (not just a sample).

In [ ]:
goldBugTokensLowercase = nltk.word_tokenize(goldBugString.lower())
goldBugRealWordTokensLowercase = [word for word in goldBugTokensLowercase if any(c.isalpha() for c in word)]
goldBugRealWordFrequencies = nltk.FreqDist(goldBugRealWordTokensLowercase)

## Finding the number of unique words

One simple way of measuring the vocabulary richness of an author is to calculate the ratio of the total number of words and the number of unique words. If an author repeats words more often, it may be because he or she is drawing on a smaller vocabulary (either deliberately or not), which is a measure of style. There are several factors to consider, such as the length of the text, but in the simplest terms a we can calculate the lexical diversity of an author by diving the number word forms (types) by the total number of tokens. We already have the necessary ingredients:

* types: number of different words (number of word: count pairs in ```goldBugRealWordFrequencies```)
* tokens: total number of word tokens (length of ```goldBugRealWordTokensLowercase```

In [ ]:
print("number of types: ", len(goldBugRealWordFrequencies))
print("number of tokens: ", len(goldBugRealWordTokensLowercase))
print("type/token ratio: ", len(goldBugRealWordFrequencies)/len(goldBugRealWordTokensLowercase))

This gives us a measure of the ratio of unique words to overall words, and it may be interesting to compare this between stories of different genre, or between early and late periods in Poe's writing.

However, we haven't yet looked at our output for the top frequency lowercase words.

In [ ]:
goldBugRealWordFrequencies.tabulate(20) # show a sample of the top frequency terms

We tokenized, filtered and counted in three lines of code, and then a fourth to show the top frequency terms, but the results aren't necessarily very exciting. There's not much in these top frequency words that could be construed as especially characteristic of _The Gold Bug_, in large part because the most frequent words are similar for most texts of a given language: they're so-called function words that have more of a syntactic (grammatical) function rather than a semantic (meaning-bearing) value.

Fortunately, our NLTK library contains a list of stopwords for English (and other languages). We can load the list and look at its contents.

In [ ]:
import nltk
stopwords = nltk.corpus.stopwords.words("english")
print(sorted(stopwords))

We can test whether one word is an item in another list with the following syntax.

In [ ]:
print("sample words: ", goldBugRealWordTokensLowercaseSample) #using our small sample
print("sample words not in stopwords list: ", [word for word in goldBugRealWordTokensLowercaseSample if not word in stopwords])

So we can now tweak our word filter with an additional condition, adding the ```and``` operator between the test for alphabetic characters and the test for presence in the stopword list. We add a slash (```\```) character to treat the code as if it were on one line. Alternatively, we could have done this in two steps (perhaps less efficient but arguably easier to read):

```python
# first filter tokens with alphabetic characters
goldBugRealContentWordTokensLowercase = [word for word in goldBugTokensLowercase if any(c.isalpha() for c in word)]
# then filter stopwords
goldBugRealContentWordTokensLowercase = [word for word in goldBugRealContentWordTokensLowercase if word not in stopwords]```

In [ ]:
goldBugRealContentWordTokensLowercase = [word for word in goldBugTokensLowercase \
        if any(c.isalpha() for c in word) and word not in stopwords]
goldBugRealContentWordFrequencies = nltk.FreqDist(goldBugRealContentWordTokensLowercase)
goldBugRealContentWordFrequencies.tabulate(30) # show a sample of the top 

Now we have words that seem a bit more meaningful (even if the table format is a bit off). The first word ("upon") could be considered a function word (a preposition) that should be in the stopword list, though it's less common in modern English. The second word ("de") would be in a French stopword list, but seems striking here in English. The third word "'s" is actually an artifact of possessive forms – sometimes tokenization keeps possessives together with the word, sometimes not. The next words ("jupiter" and "legrand") merit closer inspection, they may be proper names that have been transformed to lowercase. We can continue on like this with various observations and hypotheses, but really we probably want to have a closer look at individual occurences to see what's happening. For that, we'll build a concordance.

## Building a Simple Concordance

A concordance allows us to see each occurrence of a term in its context. It has a rich history in textual scholarship, dating back to well before the advent of computers. It's a tool for studying word usage in context.

The easiest way to build a concordance is to create an NLTK Text object from a list of word tokens (in this case we'll use the unfiltered list so that we can better read the text). So, for instance, we can ask for a concordance of "de" to try to better understand why it occurs so often in this English text.

In [ ]:
goldBugText = nltk.Text(goldBugTokens)
goldBugText.concordance("de", lines=10)

## Next Steps

Here are some tasks to try:

* show a table of the top 20 words
    * choose 3 words to add to the stopwords list using list concatenation
    * regenerate the list of the top 20 words using your new stopwords list
* instead of testing for presence in the stopword list, how would test for words that contain 10 characters or more?
* determine whether or not the word provided to the concordance function is case sensitive
* try out the NLTK sentence tokenizer (```sent_tokenize()```) on the example we tried to tokenize using the ```split()``` function in the [Getting Started](GettingStarted.ipynb) notebook ("Many years ago, I contracted an intimacy with a Mr. William Legrand..." etc)

In the next notebook we're going to introduce how to produce charts.

---
From [The Art of Literary Text Analysis](ArtOfLiteraryTextAnalysis.ipynb) by [Stéfan Sinclair](http://stefansinclair.name) &amp; [Geoffrey Rockwell](http://geoffreyrockwell.com), [CC BY-SA](https://creativecommons.org/licenses/by-sa/4.0/)<br />
Created January 19, 2015, last modified February 7, 2015